In [ ]:
import requests
import json
import pandas as pd
import time

In [ ]:
# codes for qiushuang: token-balances

# the "address" is changeable according to mapping
address = "0x7a9E457991352F8feFB90AB1ce7488DF7cDa6ed5"

# refers to https://docs.amberdata.io/reference#get-token-balances
url = "https://web3api.io/api/v2/addresses/" + address + "/token-balances/historical"

headers = {
    "Accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": "UAK2c264f10f6d17e9ba6ed2577d15c5dc1"
}

# here, the time format should be milliseconds
current_time = int(time.time() * 1000)
page = 0
while True:
    querystring = {"page": page,
                   "size": "1000",
                   "startDate": current_time - 60000,
                   "endDate": current_time,
                   "tokenAddress": "0x054f76beed60ab6dbeb23502178c52d6c5debe40"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200 and response.json()['status'] == 200:
        df = pd.json_normalize(response.json()['payload']['records'])
        with open('./Data/token_balances.csv', 'a') as f:
            df.to_csv(f, index=False, header=f.tell()==0)
        if len(response.json()['payload']['records']) == 1000:
            page += 1
        else:
            break

FileNotFoundError: ignored

In [ ]:
# codes for yinuo (1): token-transfers

# refers to https://docs.amberdata.io/reference#transaction-token-transfers
url = "https://web3api.io/api/v2/addresses/0x7a9E457991352F8feFB90AB1ce7488DF7cDa6ed5/token-transfers"

headers = {
    "Accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": "UAK2c264f10f6d17e9ba6ed2577d15c5dc1"
}

# here, the time format should be milliseconds
current_time = int(time.time() * 1000)
page = 0
while True:
    querystring = {"page": page,
                   "size": "1000",
                   "startDate": current_time - 60000,
                   "endDate": current_time,
                   "format": "csv"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200:
        if response.content != b'':
            with open('./Data/token_transfers.csv','ab') as f:
                f.write(response.content)
            page += 1
        else:
            break

In [ ]:
#############################################################################################################

# codes (2) for yinuo (for real-time update): methods type

# refers to https://docs.amberdata.io/reference#get-all-transactions
url = "https://web3api.io/api/v2/transactions"

headers = {
    "Accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": "UAK2c264f10f6d17e9ba6ed2577d15c5dc1"
}

# here, the time format should be milliseconds
current_time = int(time.time() * 1000)
page = 0
df = pd.DataFrame()
while True:
    querystring = {"startDate": current_time - 60000,
                    "endDate": current_time,
                    "size": "1000",
                    "page": page}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code == 200 and response.json()['status'] == 200:
        df = df.append(pd.json_normalize(response.json()['payload']['records']))
        if len(response.json()['payload']['records']) == 1000:
            page += 1
        else:
            break

methods_dict = {'0x4b8a3529': 'Borrow',
                '0x47e7ef24': 'Deposit',
                '0xf3fef3a3': 'Withdraw',
                '0xfa09e630': 'Withdraw', # actual for Withdrawal
                '0x22867d78': 'Repay'}

hash_list = []
methods = []

for index, row in df.iterrows():
    # drop the data not belongs to Definer
    if row['from'] is None or row['to'] is None:
        continue
    if row['from'][0]['address'] != '0x7a9E457991352F8feFB90AB1ce7488DF7cDa6ed5' and row['to'][0]['address'] != '0x7a9E457991352F8feFB90AB1ce7488DF7cDa6ed5':
        continue
    method_code = row['input'][:10]
    if method_code in methods_dict:
        hash_list.append(row['blockHash'])
        methods.append(methods_dict[method_code])
    break
    
new_df = pd.DataFrame()
new_df = new_df.assign(hash = hash_list)
new_df = new_df.assign(Methods = methods)

# save the file
with open('./Data/methods_type.csv', 'a') as f:
    new_df.to_csv(f, index=False, header=f.tell()==0)

FileNotFoundError: ignored